In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import boxball_loader as bbl
import baseball_stats_utils as bsu

In [2]:
bat = bbl.load_batting(coalesce_type=bbl.CoalesceMode.PLAYER_SEASON_LEAGUE, player_types=bbl.PlayerType.POSITION)
bat

ab    r    h  _2b  _3b  hr    rbi    sb   cs  bb    so  \
player_id yr   lg_id                                                            
aaronha01 1954 NL     468   58  131   27    6  13   69.0   2.0  2.0  28  39.0   
          1955 NL     602  105  189   37    9  27  106.0   3.0  1.0  49  61.0   
          1956 NL     609  106  200   34   14  26   92.0   2.0  4.0  37  54.0   
          1957 NL     615  118  198   27    6  44  132.0   1.0  1.0  57  58.0   
          1958 NL     601  109  196   34    4  30   95.0   4.0  1.0  59  49.0   
...                   ...  ...  ...  ...  ...  ..    ...   ...  ...  ..   ...   
zuvelpa01 1991 AL       0    0    0    0    0   0    0.0   0.0  0.0   0   0.0   
zwilldu01 1910 AL      87    7   16    5    0   0    5.0   1.0  0.0  11  16.0   
          1914 FL     592   91  185   38    8  16   95.0  21.0  0.0  46  68.0   
          1915 FL     548   65  157   32    7  13   94.0  24.0  0.0  67  65.0   
          1916 NL      53    4    6    1    0   1    8.0   0.0  0.0   4   6.0   

                       ibb  hbp    sh   sf  gidp  
player_id yr   lg_id                              
aaronha01 1954 NL      0.0  3.0   6.0  4.0  13.0  
          1955 NL      5.0  3.0   7.0  4.0  20.0  
          1956 NL      6.0  2.0   5.0  7.0  21.0  
          1957 NL     15.0  0.0   0.0  3.0  13.0  
          1958 NL     16.0  1.0   0.0  3.0  21.0  
...                    ...  ...   ...  ...   ...  
zuvelpa01 1991 AL      0.0  0.0   0.0  0.0   0.0  
zwilldu01 1910 AL      0.0  1.0   1.0  0.0   0.0  
          1914 FL      0.0  1.0  10.0  0.0   0.0  
          1915 FL      0.0  2.0  18.0  0.0   0.0  
          1916 NL      0.0  0.0   2.0  0.0   0.0  

[58093 rows x 16 columns]

In [3]:
ppl = bbl.load_people()[['player_id', 'bats', 'throws']]
ppl

,player_id,bats,throws
0,aardsda01,R,R
1,aaronha01,R,R
2,aaronto01,R,R
3,aasedo01,R,R
4,abadan01,L,L
...,...,...,...
20088,zupofr01,L,R
20089,zuvelpa01,R,R
20090,zuverge01,R,R
20091,zwilldu01,L,L


In [4]:
hr_bat = bsu.add_batting_rate_stats(bat)[['pa', 'hr']] # add_batting_rate_stats will compute pa
bat_seasons = pd.merge(left=hr_bat.reset_index(), right=ppl, on='player_id')
bat_seasons

,player_id,yr,lg_id,pa,hr,bats,throws
0,aaronha01,1954,NL,503.0,13,R,R
1,aaronha01,1955,NL,658.0,27,R,R
2,aaronha01,1956,NL,655.0,26,R,R
3,aaronha01,1957,NL,675.0,44,R,R
4,aaronha01,1958,NL,664.0,30,R,R
...,...,...,...,...,...,...,...
58088,zuvelpa01,1991,AL,0.0,0,R,R
58089,zwilldu01,1910,AL,99.0,0,L,L
58090,zwilldu01,1914,FL,639.0,16,L,L
58091,zwilldu01,1915,FL,617.0,13,L,L


In [5]:
bat_totals = bat_seasons.groupby(['yr', 'bats'])[['pa', 'hr']].sum()
bat_rolling_totals = bat_totals.unstack(level=1).rolling(5).sum().dropna()
bat_rolling_totals


pa                          hr                 
bats         B         L         R       B       L        R
yr                                                         
1880    1881.0   18521.0   61522.0     3.0    64.0    131.0
1881    2714.0   21506.0   62797.0     6.0    74.0    151.0
1882    3233.0   28145.0   76246.0    10.0   112.0    230.0
1883    4532.0   38179.0  100251.0    14.0   159.0    365.0
1884    7944.0   54437.0  143932.0    30.0   283.0    771.0
...        ...       ...       ...     ...     ...      ...
2016  118947.0  303482.0  467178.0  2613.0  8101.0  13477.0
2017  118806.0  299703.0  473235.0  2730.0  8399.0  14230.0
2018  119392.0  293463.0  479902.0  2908.0  8480.0  14893.0
2019  117905.0  289908.0  488277.0  3168.0  9263.0  16431.0
2020  103204.0  251954.0  429793.0  2924.0  8319.0  15039.0

[141 rows x 6 columns]

In [6]:
bat_rolling_rates = bat_rolling_totals['hr']/bat_rolling_totals['pa']
bat_rolling_rates

bats,B,L,R
yr,,,
1880,0.001595,0.003456,0.002129
1881,0.002211,0.003441,0.002405
1882,0.003093,0.003979,0.003017
1883,0.003089,0.004165,0.003641
1884,0.003776,0.005199,0.005357
...,...,...,...
2016,0.021968,0.026694,0.028848
2017,0.022979,0.028024,0.030070
2018,0.024357,0.028896,0.031033


In [7]:
fig = px.line(bat_rolling_rates[['L', 'R', 'B']].stack().rename('rate').reset_index(), x='yr', y='rate', color='bats')
fig.update_layout(width=800)
fig.write_image('output/hr_rates_by_handedness.png')
fig.show()

In [8]:
bat_rolling_ratios = (bat_rolling_rates['L']/bat_rolling_rates['R']).rename('HR Rate LHB:RHB')
bat_rolling_ratios

yr
1880    1.622836
1881    1.430982
1882    1.319186
1883    1.143848
1884    0.970499
          ...   
2016    0.925326
2017    0.931984
2018    0.931135
2019    0.949497
2020    0.943605
Name: HR Rate LHB:RHB, Length: 141, dtype: float64

In [9]:
fig = px.line(bat_rolling_ratios)
fig.update_layout(width=800)
fig.write_image('output/hr_rates_by_handedness_ratio.png')
fig.show()

In [10]:
# Out of curiosity, how much did the mix of playing time by handedness change
pa_totals = bat_totals['pa'].unstack(level=1)
pa_totals_rolling = pa_totals.rolling(5).sum().dropna()
pa_sum_rolling = pa_totals_rolling.sum(axis=1)
pa_fractions = pa_totals_rolling.divide(pa_sum_rolling, axis=0)

fig = px.line(pa_fractions[['L', 'R', 'B']])
fig.update_layout(width=800)
fig.write_image('output/pa_by_handedness.png')
fig.show()


In [11]:
# Is there any NL/AL effect?  Look at LHB/RHB ratios for each league
alnl = bat_seasons[(bat_seasons['lg_id'].isin(['AL', 'NL']))&(bat_seasons['yr']>1900)]
bat_totals = alnl.groupby(['yr', 'bats', 'lg_id'])[['pa', 'hr']].sum()
bat_rolling_totals = bat_totals.unstack().unstack().rolling(5).sum().dropna()
bat_rolling_rates = bat_rolling_totals['hr']/bat_rolling_totals['pa']
bat_rolling_rates

lg_id        AL                            NL                    
bats          B         L         R         B         L         R
yr                                                               
1905   0.003661  0.004694  0.005248  0.003096  0.004811  0.003655
1906   0.002969  0.004284  0.004651  0.002364  0.004032  0.003437
1907   0.002000  0.003501  0.003676  0.002246  0.004127  0.003768
1908   0.001949  0.003231  0.003165  0.002413  0.003955  0.003775
1909   0.001756  0.003059  0.002926  0.002330  0.003839  0.003619
...         ...       ...       ...       ...       ...       ...
2016   0.023091  0.027704  0.030243  0.020281  0.025692  0.027511
2017   0.024252  0.028601  0.031275  0.021035  0.027467  0.028843
2018   0.025501  0.029007  0.032209  0.022759  0.028794  0.029793
2019   0.027573  0.032185  0.034860  0.025930  0.031738  0.032354
2020   0.028730  0.033203  0.035825  0.027824  0.032853  0.034081

[116 rows x 6 columns]

In [12]:
# Visualize each league's trend (I tried putting it all on one chart, but it was too busy)
for lg in ('AL', 'NL'):
    fig = px.line(bat_rolling_rates[lg][['L', 'R']], labels={'value': 'HR rate (5-year rolling)'})
    fig.update_layout(width=800, title=f"HR Rate by handedness ({lg})")
    fig.write_image(f'output/hr_rates_by_handedness_{lg}.png')
    fig.show()

# Looks like the LH/RH effect from 1910-1950 was strong in both leagues

In [13]:
# OK, let's look at the distribution of HR hitters.  Look by cohort, etc

# This will filter the sample by handedness and then split into deciles
# Used by compare_deciles()
def get_deciles(exploded, bats):
    df = exploded.query('bats==@bats')
    bins = pd.qcut(df['hr_rate'],10)
    return(df.groupby(bins)['hr_rate'].mean())

# This will generate the decile summary for a given set of years
def compare_deciles(years):
    # Get the raw stats, then add PA, handedness and HR rate
    raw = bbl.load_batting(years, coalesce_type=bbl.CoalesceMode.PLAYER_CAREER, player_types=bbl.PlayerType.POSITION)
    hr_bat = bsu.add_batting_rate_stats(raw)[['pa', 'hr']] # add_batting_rate_stats will compute pa
    bat_careers = pd.merge(left=hr_bat.reset_index(), right=ppl, on='player_id')
    bat_careers['hr_rate'] = bat_careers['hr']/bat_careers['pa']
    
    # To weight by PA, repeat each line once per PA
    exploded = pd.DataFrame(bat_careers.values.repeat(bat_careers['pa'], axis=0), columns=bat_careers.columns)[['player_id', 'bats', 'hr_rate']]
    exploded['hr_rate'] = exploded['hr_rate'].apply(float)

    # Now compute the summary means
    by_handed = pd.concat([get_deciles(exploded, bats).rename(bats).reset_index(drop=True) for bats in ('L', 'R')], axis=1)
    return by_handed

deciles = compare_deciles(range(1910, 1951))
deciles


,L,R
0,0.001291,0.000433
1,0.003486,0.001959
2,0.005409,0.003290
3,0.007794,0.004683
4,0.009642,0.006312
5,0.011599,0.008292
6,0.013871,0.010883
7,0.018294,0.015215
8,0.026157,0.020968
9,0.042175,0.036621


In [14]:

for yrs in (range(1910, 1947), range(1947, 1970), range(1970, 2021)):
    comparison = compare_deciles(yrs)
    fig = px.line(comparison, labels={'value': f'Cohort Career HR Rate', 'index': 'decile'})
    fig.update_layout(width=800, title=f"HR Rate by handedness and decile ({yrs.start}-{yrs.stop-1})")
    fig.write_image(f'output/hr_rates_by_cohort_{yrs.start}-{yrs.stop-1}.png')
    fig.show()


In [15]:
deciles['L']/deciles['R']

0    2.982037
1    1.779360
2    1.644428
3    1.664327
4    1.527625
5    1.398847
6    1.274530
7    1.202367
8    1.247480
9    1.151683
dtype: float64